# Data Preprocessing Overview
Data preprocessing is critical to ensure the dataset is clean, consistent, and informative, maximizing the accuracy of the analytics we perform. This dataset's preprocessing will cover several essential steps: handling missing values, standardizing categorical data, extracting IP information, time formatting, and scaling. Each step aims to improve the quality of data for effective alerting and policy enforcement.

### Handling Missing Values:
Missing data, especially in critical fields, can compromise insights. For this step, we'll handle nulls appropriately based on column type and purpose.

### Feature Extraction:
Some columns, like Source and Target, contain JSON-like nested information. We'll extract relevant information, such as IPs and ports, to analyze traffic patterns and anomalies.

### Encoding Categorical Variables:
Categorical data like Category and Format need encoding so that the machine learning models can interpret them correctly.

### DateTime Formatting:
The DetectTime, WinStartTime, and WinEndTime columns contain timestamps. Properly formatting and aligning these will allow for time-based analyses, such as detecting unusual access times.

### Scaling Numeric Features: 
Features such as ByteCount, PacketCount, and FlowCount have different ranges. Scaling these values can improve the model’s performance by giving equal importance to all numeric features.



## Importing Libaries


In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np

## Load the Dataset

In [16]:
file_path = 'C:/Users/USER/UEBA_Project/alerting_policy_enforcement/data/raw/alert_policy_sample.csv'
df = pd.read_csv(file_path)

## Handle Missing Values
We’ll inspect the dataset to understand the extent of missing values and choose how to handle them based on their significance. Columns like ByteCount, PacketCount, and EventTime with many missing values may require imputation or removal, depending on their importance.

In [17]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)

# Drop columns with a high percentage of missing data if they aren't critical
df.drop(columns=['ByteCount', 'EventTime', 'CreateTime'], inplace=True)

# Impute missing values in other columns if necessary
df['ConnCount'].fillna(df['ConnCount'].median(), inplace=True)
df['FlowCount'].fillna(0, inplace=True)
df['PacketCount'].fillna(df['PacketCount'].mean(), inplace=True)


Missing Values:
 ByteCount       496660
Category             0
CeaseTime       215707
ConnCount        75296
CreateTime      264616
DetectTime           0
EventTime       211508
FlowCount       264629
Format               0
ID                   0
Node                 0
PacketCount     496718
Source            2007
Target          172823
WinEndTime      395413
WinStartTime    395413
dtype: int64


C:\Users\USER\AppData\Local\Temp\ipykernel_7284\2847865510.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ConnCount'].fillna(df['ConnCount'].median(), inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_7284\2847865510.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop

## Convert Time Columns to DateTime Format
Convert columns like DetectTime, WinEndTime, and WinStartTime to DateTime format for better time-based analysis.

In [18]:
# Convert time columns to DateTime format
time_columns = ['DetectTime', 'WinEndTime', 'WinStartTime']
for col in time_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Check for any issues in the conversion
print(df[time_columns].head())


C:\Users\USER\AppData\Local\Temp\ipykernel_7284\948374484.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\USER\AppData\Local\Temp\ipykernel_7284\948374484.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\USER\AppData\Local\Temp\ipykernel_7284\948374484.py:4: FutureWarning: In a future version of pandas, parsing datetime

                  DetectTime WinEndTime WinStartTime
0  2019-03-11 00:05:28+00:00        NaT          NaT
1  2019-03-11 00:05:21+00:00        NaT          NaT
2  2019-03-11 00:05:30+00:00        NaT          NaT
3  2019-03-11 00:05:58+00:00        NaT          NaT
4  2019-03-11 00:05:58+00:00        NaT          NaT


## Extract Features from Time Columns
Using time-based columns, we can extract additional features like the hour, day, or week to capture temporal patterns in the data.

In [19]:
# Convert columns to datetime format
df['DetectTime'] = pd.to_datetime(df['DetectTime'], errors='coerce')
df['WinEndTime'] = pd.to_datetime(df['WinEndTime'], errors='coerce')
df['WinStartTime'] = pd.to_datetime(df['WinStartTime'], errors='coerce')

# Extract additional time-based features
df['DetectHour'] = df['DetectTime'].dt.hour
df['DetectDay'] = df['DetectTime'].dt.day
df['WinDuration'] = (df['WinEndTime'] - df['WinStartTime']).dt.total_seconds()


## Feature Extraction from JSON-like Columns

In [ ]:

# Define helper function to safely parse JSON-like data
def extract_field(data, field, nested_field=None):
    try:
        if pd.notnull(data):
            parsed_data = ast.literal_eval(data)
            if isinstance(parsed_data, list) and len(parsed_data) > 0:
                if nested_field and nested_field in parsed_data[0]:
                    return parsed_data[0][nested_field][0] if field in parsed_data[0] else None
                return parsed_data[0][field][0] if field in parsed_data[0] else None
    except (ValueError, SyntaxError, IndexError, KeyError):
        return None
    return None

# Extract fields from `Node`, `Source`, and `Target`
df['Node_SW'] = df['Node'].apply(lambda x: extract_field(x, 'SW'))
df['Node_Type'] = df['Node'].apply(lambda x: extract_field(x, 'Type'))
df['Source_Proto'] = df['Source'].apply(lambda x: extract_field(x, 'Proto'))
df['Source_Port'] = df['Source'].apply(lambda x: extract_field(x, 'Port'))
df['Target_Proto'] = df['Target'].apply(lambda x: extract_field(x, 'Proto'))
df['Target_Port'] = df['Target'].apply(lambda x: extract_field(x, 'Port'))

## Encode Categorical Variables
For columns like Category and Format, we can use one-hot encoding or label encoding for machine learning compatibility.

In [20]:
import joblib
encoder = LabelEncoder()
df['Category_encoded'] = encoder.fit_transform(df['Category'])
joblib.dump(encoder, 'category_encoder.pkl')


# Use one-hot encoding for columns with multiple categorical values
ohe = OneHotEncoder(sparse=False)
encoded_proto = ohe.fit_transform(df[['Source_Proto', 'Target_Proto']].fillna('missing'))
encoded_proto_df = pd.DataFrame(encoded_proto, columns=ohe.get_feature_names_out(['Source_Proto', 'Target_Proto']))
df = pd.concat([df.reset_index(drop=True), encoded_proto_df], axis=1)
joblib.dump(ohe, 'proto_onehot_encoder.joblib')

['category_encoder.pkl']

## Scale Numerical Features
To standardize numerical columns, scaling them can improve model performance, especially for algorithms sensitive to feature ranges.

In [21]:
scaler = StandardScaler()
numerical_cols = ['ConnCount', 'FlowCount', 'PacketCount', 'WinDuration']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
joblib.dump(scaler, 'scaler.joblib')


## Save Preprocessed Data**

In [22]:

# Save the preprocessed DataFrame to a new CSV file
processed_file_path = 'C:/Users/USER/UEBA_Project/alerting_policy_enforcement/data/processed/alert_policy_preprocessed.csv'
df.to_csv(processed_file_path, index=False)

print("Preprocessing complete. Label encoder and scaler saved.")

Preprocessing complete. Label encoder and scaler saved.


In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import joblib
import ast

# Load the dataset
file_path = 'C:/Users/USER/UEBA_Project/alerting_policy_enforcement/data/raw/alert_policy_sample.csv'
df = pd.read_csv(file_path)

# **Step 1: Handle Missing Values**
df.dropna(subset=['ID'], inplace=True)
# Update to avoid chained assignment warning
for col in ['ByteCount', 'ConnCount', 'FlowCount', 'PacketCount']:
    df[col] = df[col].fillna(df[col].median())

# **Step 2: Feature Extraction from JSON-like Columns**
# Define helper function to safely parse JSON-like data
def extract_field(data, field, nested_field=None):
    try:
        if pd.notnull(data):
            parsed_data = ast.literal_eval(data)
            if isinstance(parsed_data, list) and len(parsed_data) > 0:
                if nested_field and nested_field in parsed_data[0]:
                    return parsed_data[0][nested_field][0] if field in parsed_data[0] else None
                return parsed_data[0][field][0] if field in parsed_data[0] else None
    except (ValueError, SyntaxError, IndexError, KeyError):
        return None
    return None

# Extract fields from `Node`, `Source`, and `Target`
df['Node_SW'] = df['Node'].apply(lambda x: extract_field(x, 'SW'))
df['Node_Type'] = df['Node'].apply(lambda x: extract_field(x, 'Type'))
df['Source_Proto'] = df['Source'].apply(lambda x: extract_field(x, 'Proto'))
df['Source_Port'] = df['Source'].apply(lambda x: extract_field(x, 'Port'))
df['Target_Proto'] = df['Target'].apply(lambda x: extract_field(x, 'Proto'))
df['Target_Port'] = df['Target'].apply(lambda x: extract_field(x, 'Port'))

# **Step 3: Encode Categorical Columns**
# Encode 'Category' and save the encoder
encoder = LabelEncoder()
df['Category_encoded'] = encoder.fit_transform(df['Category'])
joblib.dump(encoder, 'C:/Users/USER/UEBA_Project/alerting_policy_enforcement/data/processed/category_encoder.joblib')

# Use one-hot encoding for columns with multiple categorical values
ohe = OneHotEncoder(sparse_output=False)  # Update `sparse` to `sparse_output`
encoded_proto = ohe.fit_transform(df[['Source_Proto', 'Target_Proto']].fillna('missing'))
encoded_proto_df = pd.DataFrame(encoded_proto, columns=ohe.get_feature_names_out(['Source_Proto', 'Target_Proto']))
df = pd.concat([df.reset_index(drop=True), encoded_proto_df], axis=1)
joblib.dump(ohe, 'C:/Users/USER/UEBA_Project/alerting_policy_enforcement/data/processed/proto_onehot_encoder.joblib')

# **Step 4: Scale Numerical Features**
scaler = StandardScaler()
numerical_cols = ['ConnCount', 'FlowCount', 'PacketCount']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
joblib.dump(scaler, 'C:/Users/USER/UEBA_Project/alerting_policy_enforcement/data/processed/scaler.joblib')

# **Step 5: Save Preprocessed Data**
processed_file_path = 'C:/Users/USER/UEBA_Project/alerting_policy_enforcement/data/processed/alert_policy_preprocessed.csv'
df.to_csv(processed_file_path, index=False)

print("Preprocessing complete. Label encoder, one-hot encoder, and scaler saved.")


Preprocessing complete. Label encoder, one-hot encoder, and scaler saved.
